# Simulation numérique – ProgFest 2025
---
Par Arthur Légaré, le 5 mars 2025.

*Le masculin est utilisé pour alléger le texte.*

# Pourquoi simuler en physique?
---

**Le carré de sable est à l'enfant ce que la simulation est au physicien.**

Curieux de nature, le physicien est porté à expérimenter avec toutes sortes de cossins à sa disposition. Toutefois, avec la maîtrise du langage mathématique, sa curiosité vient généralement à bout des expériences immédiatement accessibles : il devient donc urgent d'agrandir le carré de sable!

Venons-y : la physique comporte son lot de phénomènes qui sont décrits par des équations complexes, souvent difficiles, voire impossibles, à résoudre analytiquement. C'est là que la simulation numérique entre en jeu, notamment pour :
1) explorer des systèmes trop complexes pour admettre une solution exacte;
2) visualiser des dynamiques évolutives et leurs comportements émergents;
3) tester des hypothèses et comparer avec des résultats expérimentaux.

Certains puristes pourraient y voir là les contours d'une sacrification de la physmath sur l'autel de la *force brute*. Mais attention : l'usage des simulations ne relègue pas aux oubliettes la physmath. Au contraire, nous verrons plutôt que cette dernière demeure l'outil privilégié pour affuter notre intuition, guidant ainsi nos explorations numériques et facilitant l'interprétation des résultats.


# 1. Plan de match
---

Nous verrons trois exemples de simulations numériques, chacune illustrant un aspect clé de la modélisation en physique.

**a) Dynamique écologique** – Modélisation de la coopération entre espèces et ses effets sur taux de survie.

**b) Échanges thermiques dans un bâtiment** – Comparaison entre différentes stratégies de chauffage.

**c) Modèle d’Ising en 2D** – Étude des transitions de phase dans un système de spins.


<span style="color:red">*titres accrocheurs : a) good guys really finish least? b) on chauffe pas le deyors c) vitre funky *</span>.

# 2. Comment utiliser ce notebook
---
* Chaque simulation sera introduite par une brève présentation du problème et des équations associées.

* Le code principal est externalisé dans des fichiers Python (.py), améliorant la modularité du code. Ceci permet facilite ainsi l'exploration approfondie après l’atelier et l'implémentation de nouvelles simulations!

# 3. Simulations en rafales
---

In [ ]:
# Importons quelques librairies utiles.
import numpy as np
import matplotlib.pyplot as plt
import skimage

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['font.size'] = 16

---
## a) Coopération entre espèces
<img src="figures/lapino_chaos.png" width=800 height=300 />

### Deux espèces en présence :
TFT et rapace. en proportions différentes. 

#### Application de l'inégalité de Nash.
(Notions d'écologie et de théorie du jeu)

### Version simplifiée
d'un cas particulier dans le simulateur mis sur pied en Dynamique non linéaire et chaos (présenté au sympsium 2024). 

### Conclusions : 
qu'est-ce qui supporte l'émergence de comportements altruistes dans la nature. 

### Pour pousser plus loin

lectures : le gène égoïste, vidéo de Veritasium, article d'Axelrod. Projet de Chaos.

---
## b) Échanges thermiques dans un bâtiment

<img src="figures/barre.png" width=650 height=300 />

### Problématique : ma coloque l'an passé avait l'habitude de descendre son chauffage au minimum et de fermer sa porte à son départ. 
Mais y a-t-il une durée critique à partir de laquelle ça vaut la peine d'effectuer une telle chose? Quand on s'éloigne d'un équilibre, le retour sort du régime de linéarité et engendre possiblement une moins bonne efficacité énergétique. Il faut vérifier cette hypothèse en balayant la durée pour une configuration vraisemblable.

Équations de thermodynamique parachutées : vous devez me faire confiance.

---
## c) Modèle d'Ising en 2D

### Phénoménologie selon la température.

### Visualisation
des patrons au seuil critique de changement de phase

### Modèle simplifié pour approcher le ferromagnétisme

### Finale :
qu'arrive-t-il si on introduit un champ magnétique. Réponse : revenez me voir dans quelques moi, je vais étudier le sujet de manière plus exhaustive.

# 4. Conclusion : Et si les simulations apprenaient d'elles-mêmes ?

Nous avons illustré au moyen de trois exemples en quoi les simulations numériques permettent de modéliser des systèmes complexes en physique. Mais que se passerait-il si, au lieu de coder explicitement chaque interaction, on laissait un modèle apprendre ces dynamiques?

C’est là qu’interviennent les réseaux neuronaux entraînés sur des modèles physiques. Plutôt que d’évaluer chaque équation pas à pas, on entraîne un réseau sur des données simulées. Une fois entraîné, le réseau tente de prédire l’évolution du système bien plus rapidement qu’une intégration numérique classique. 

En théorie, cette méthode est loin d'être farfelue, puisqu'on sait depuis 1989 [1] qu'un réseau neuronal peut agir à titre d'« approximateur universel » de toutes fonctions continues! Il est donc logique d'essayer de capturer la dynamique d'un système de cette manière.

Cette approche commence déjà à être utilisée pour accélérer les simulations en mécanique des fluides, en physique des matériaux, et même en cosmologie! <span style="color:red">sources</span>.


**Les PINNs : le bac à sable du futur**

Les simulations basées sur les équations (comme celles du présent atelier) sont complémentaires à celles basées sur l’apprentissage machine. En combinant la rigueur des modèles physiques avec la puissance des réseaux neuronaux, on ouvre la voie à des simulations 
1) plus rapides,
2) moins énergivores,
3) détenant la capacité de généraliser sur de nouveaux cas.
   
On parle alors de « réseaux de neurones informés par la physique », ou PINNs (*Physics-informed neural networks*) en anglais. <span style="color:red">sources</span>

# 5. Références
[1] G. Cybenko, “Approximation by superpositions of a sigmoidal function,” Math. Control Signal Systems, vol. 2, no. 4, pp. 303–314, Dec. 1989, doi: 10.1007/BF02551274.

[2] “Loi de refroidissement de Newton,” Wikipédia. Jul. 29, 2023. Accessed: Mar. 02, 2025. [Online]. Available: https://fr.wikipedia.org/w/index.php?title=Loi_de_refroidissement_de_Newton&oldid=206457504

[3] “Matrice laplacienne,” Wikipédia. Sep. 15, 2024. Accessed: Mar. 02, 2025. [Online]. Available: https://fr.wikipedia.org/w/index.php?title=Matrice_laplacienne&oldid=218663112

[4] E. W. Weisstein, “Positive Definite Matrix.” Accessed: Mar. 02, 2025. [Online]. Available: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html

[5] “Thermal conduction,” Wikipedia. Feb. 25, 2025. Accessed: Mar. 02, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Thermal_conduction&oldid=1277574658

[6] I. Newton, “VII. Scala graduum caloris,” in Philosophical Transactions of the Royal Society of London, vol. 22, The Royal Society, 1701, pp. 824–829. doi: 10.1098/rstl.1700.0082.




# 6. Bonus 

## Recette : Comment réussir (ou pas) une simulation numérique

1. **Trouver un problème :** arroser les fleurs du tapis de la curiosité intellectuelle.
2. **Mathématiser le problème :** *soit un boeuf sphérique précédé d'une charrue...*
3. **Choisir des paramètres :** [*Itération 1*] partons à la cueillette de cerises, sans vergogne.
4. **Écrire une belle architecture** de code (digne du brutalisme, avec juste ce qu'il faut de chaos).
5. **Coder**, c’est-à-dire s'adonner à la chasse aux bugs en écoutant du [Georges Brassens](https://www.youtube.com/watch?v=oOPBnbNxLDM).
6. **Visualiser les résultats** (et ajuster les couleurs pour que ça ait l’air crédible).
